In [27]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import statsmodels.tsa.vector_ar.vecm as vecm
import statsmodels.formula.api as smf
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.stattools import adfuller


## testing armington elastcities UK gas market ##

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/petrol_arm.csv'
df = pd.read_csv(url)

SyntaxError: unexpected EOF while parsing (<ipython-input-27-3d4e0e71a479>, line 19)

In [2]:
print(df)

       Date  ConsRatio  PriceRatio  ConsRatioL  ConsRatD  Q1  Q2  Q3  Q4  \
0    Mar-06  -0.803231    0.143038         NaN       NaN   1   0   0   0   
1    Jun-06  -0.740313    0.052014   -0.803231  0.062918   0   1   0   0   
2   Sept-06  -0.795364    0.167691   -0.740313 -0.055051   0   0   1   0   
3    Dec-06  -0.482442    0.217325   -0.795364  0.312922   0   0   0   1   
4    Mar-07  -0.400200    0.201160   -0.482442  0.082242   1   0   0   0   
..      ...        ...         ...         ...       ...  ..  ..  ..  ..   
59   Dec-20  -0.696740    0.455440   -0.082779 -0.613961   0   0   0   1   
60   Mar-21   0.425316    0.276913   -0.696740  1.122056   1   0   0   0   
61   Jun-21  -0.331717    0.202427    0.425316 -0.757033   0   1   0   0   
62  Sept-21  -0.336606    0.222178   -0.331717 -0.004889   0   0   1   0   
63   Dec-21  -0.745841    0.302228   -0.336606 -0.409235   0   0   0   1   

    PriceRat2  PriceRatD  
0    0.109982        NaN  
1    0.077153  -0.032829  
2    0

In [ ]:
lag_order1 = vecm.select_order(data=df[['ConsRatio', 'PriceRat2']], maxlags=5, deterministic="ci", exog=df[['Q1','Q2','Q3','Q4']])
print(lag_order1.selected_orders)

rank = vecm.select_coint_rank(endog=df[['ConsRatio', 'PriceRat2']],
                              k_ar_diff=5,
                              det_order=0)

print(rank)

In [33]:
m1 = vecm.VECM(df[['ConsRatio', 'PriceRatio']], deterministic="ci", k_ar_diff=1, coint_rank=1)  
vecm_res_1 = m1.fit()
vecm_res_1.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
L1.ConsRatio,-0.2676,0.119,-2.246,0.025,-0.501,-0.034
L1.PriceRatio,1.6056,0.483,3.322,0.001,0.658,2.553
,coef,std err,z,P>|z|,[0.025,0.975]
L1.ConsRatio,-0.0060,0.031,-0.195,0.845,-0.066,0.054
L1.PriceRatio,-0.1718,0.125,-1.376,0.169,-0.416,0.073
,coef,std err,z,P>|z|,[0.025,0.975]
ec1,-0.6011,0.171,-3.511,0.000,-0.937,-0.266
,coef,std err,z,P>|z|,[0.025,0.975]
ec1,-0.0725,0.044,-1.641,0.101,-0.159,0.014
,coef,std err,z,P>|z|,[0.025,0.975]


In [ ]:
granger_results = vecm_res_1.test_granger_causality(caused="ConsRatio", signif=0.05)
granger_results.summary()

In [37]:
PR_constant = sm.add_constant(PriceRat2)
result1 = smf.ols(formula="ConsRatio ~ PR_constant", data=df).fit()
print(result1.summary())

NameError: name 'PriceRat2' is not defined

In [15]:
test = ts.adfuller(df['PriceRat2'],autolag="AIC")
print(test)

(-2.2762490393890316, 0.17974257343235678, 4, 59, {'1%': -3.5463945337644063, '5%': -2.911939409384601, '10%': -2.5936515282964665}, -199.79911997411625)
